In [1]:
"""
this notebook is referenced from:
https://lightning.ai/docs/pytorch/stable/notebooks/course_UvA-DL/05-transformers-and-MH-attention.html
https://github.com/aceliuchanghong/LLMs-from-scratch
https://github.com/aceliuchanghong/llms-from-scratch-cn
https://github.com/aceliuchanghong/Transformer-from-scratch
"""

'\nthis notebook is referenced from:\nhttps://lightning.ai/docs/pytorch/stable/notebooks/course_UvA-DL/05-transformers-and-MH-attention.html\nhttps://github.com/aceliuchanghong/LLMs-from-scratch\nhttps://github.com/aceliuchanghong/llms-from-scratch-cn\nhttps://github.com/aceliuchanghong/Transformer-from-scratch\n'

In [20]:
# 引入依赖
# PyTorch Lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

# Standard libraries
import math
import os
import sys
from functools import partial

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import numpy as np

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

# Torchvision
import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR100
from tqdm.notebook import tqdm

# Log
from dotenv import load_dotenv
import logging
from termcolor import colored

load_dotenv()
log_level = os.getenv("LOG_LEVEL", "INFO").upper()
logging.basicConfig(
    level=getattr(logging, log_level),
    format="%(asctime)s-%(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(__name__)

In [28]:
# 超参数设置一下
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 4 # 批次大小
context_length = 16 # 文本长度 ==> 截取一段话,其最多有 context_length 个 token 
d_model = 64  # 模型维度
num_layers = 8  # 模型 layer 层数
num_heads = 4  # 通过 d_model / num_heads = 来获取 head_size
learning_rate = 1e-3  # 学习率 0.001
dropout = 0.1 # Dropout rate
max_iters = 500  # 迭代次数
eval_interval = 50  # 每多少轮验证模型 
eval_iters = 20  # 评估模型时，需要对损失进行多少次迭代平均

TORCH_SEED = 1337
torch.manual_seed(TORCH_SEED)

In [29]:
# 文件读取
with open('../../../z_using_files/txt/Reverend_Insanity.txt','r',encoding='utf-8') as f:
    text = f.read()
text[:200]

'Information\nTable of Contents URL: https://novelfull.com/reverend-insanity.html\n\n\n\nAuthor:Gu Zhen Ren, 蛊真人\nAlternative names:Cổ chân nhân, Daoist Gu, Gu Zhen Ren, 蛊真人\nGenre:Fantasy, Martial Arts, Acti'

In [30]:

# 使用 TikToken 将文本 token 化
import tiktoken
cl100k_base = tiktoken.get_encoding("cl100k_base")
enc = cl100k_base.encode(text)
tokenized_text = torch.tensor(enc, dtype=torch.long, device=device) # Convert tokens into a tensor
max_token_value = tokenized_text.max().item()
logger.info(colored(f"{len(enc)}", "green"))
logger.info(colored(f"{enc[:20]}", "green"))
logger.info(colored(f"max_token_value:{max_token_value}", "green"))

2025-01-05 17:38:39-INFO: 6581059
2025-01-05 17:38:39-INFO: [15218, 198, 2620, 315, 36962, 5665, 25, 3788, 1129, 39142, 491, 620, 916, 10991, 424, 408, 22610, 39370, 2628, 1038]
2025-01-05 17:38:39-INFO: max_token_value:100252


In [33]:
# 数据分割训练集,验证集
split_idx = int(len(tokenized_text) * 0.9)
train_data = tokenized_text[:split_idx]
val_data = tokenized_text[split_idx:]
train_data, val_data , len(train_data), len(val_data)

(tensor([15218,   198,  2620,  ...,  1053,  3719,   872]),
 tensor([91231,   481,  1853,  ...,  3284,    13,   720]),
 5922953,
 658106)